# Importation des librairies nécessaires à notre environnement
* on reprend les imports [des 6 modules Python](https://github.com/javaskater/rcp217_project/tree/main/PYTHON)

In [ ]:
# From DataLoader modules
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, IntTensor
import os
import re
import csv
# From RESNET1D and averaging modukes
import torch.nn as nn
import torch
# From project 16 module
import torch.nn.functional as TF
# From the training module
import datetime  # for timestamping our training evolution
import torch.optim as optim
# from the validation module
## standard Python
import collections

# partie DataSource creation
* reprend le code du module [Création de DataSource avec partie main pour test en environnement virtualisé local](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/dataloader_creation.py)
* L'explication correspondante est en [lien Markdown](https://github.com/javaskater/rcp217_project/blob/main/docs/PYTHON_STEPS/1-DATALOADER.md)
## TODO 
* reprendre dans [le TP10](https://jhub3.cnam.fr/user/24592/lab/tree/MonDossier/RCP217/TP10_07042025/TP02_corrige.ipynb)
```python
from sklearn.model_selection import train_test_split
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)
```
* résoudre le problème des labels en output de la DataSource cf [Question 3 posée au professeur](https://github.com/javaskater/rcp217_project/blob/main/docs/Questions/3-PYTORCH_TENSORS.md)
* Trouver le chemin Drive Google Colab pour mes répertoires de Séries Temporelles générées par R

In [ ]:
class TimeSeriesDatasetForPCoefficients(Dataset):
    def __init__(self, list_of_pathes_to_the_timeseries_files: list):
        super().__init__()
        self.X = [] # les Time series
        self.y = [] # le coefficient p correspondant
        for path_to_the_timeseries_files in list_of_pathes_to_the_timeseries_files:
            for fichier_time_serie_name in os.listdir(path_to_the_timeseries_files):
                if fichier_time_serie_name.endswith(".csv"):
                    coeff_pq_search = re.search('ar_([0-9]{1})__ma_([0-9]{1})', fichier_time_serie_name, re.IGNORECASE)
                    if coeff_pq_search:
                        p_str = coeff_pq_search.group(1)
                        p = int(p_str)
                        q_str = coeff_pq_search.group(2)
                        q = int(q_str)
                        chemin_complet_fichier_time_serie = os.path.join(path_to_the_timeseries_files, fichier_time_serie_name)
                        print(f"[TimeSeriesDatasetForPCoefficients: init] fichier trouvé {chemin_complet_fichier_time_serie} avec pattern pour p oefficient: {p} et pour q coefficient: {q}")
                        with open(chemin_complet_fichier_time_serie, mode ='r')as file:
                            csvFile = csv.reader(file, delimiter =';')
                            next(csvFile) # skip the header
                            for ligne_time_serie_str in csvFile:
                                ligne_time_serie = FloatTensor(list(map(float, ligne_time_serie_str)))
                                print(ligne_time_serie)
                                self.X.append(ligne_time_serie)
                        p_tensor = IntTensor([p]) # Ce doit être un tenseur avec une seule donnée: l'ordre attenduexit()
                        self.y.append(p_tensor)

    def __len__(self):
        assert len(self.X) == len(self.y)
        return len(self.y)
    
    def __getitem__(self, index):
        assert index < len(self.y)
        return self.X[index], self.y[index]

# Partie 2 et 3 du réseau de neuronnes
## ce module est commun 
* Etape 2 du réseau de Neuronnes: 4 convolutions RESNET 1D avec kernel de 10 et Stride de 1
* Etape 3 du réseau de Neuronnes: 6 convolutions RESNET 1D avec kernel de 1 et Stride de 1
## liens
* [le module lui même avec une partie main pour test](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/resnet1d_module.py)
* [la documentation Markdown sur ce module](https://github.com/javaskater/rcp217_project/blob/main/docs/PYTHON_STEPS/2-RESNET.md) 

In [ ]:
class ResNet1D(nn.Module):

    def __init__(self,in_channels,out_channels,kernel_size,stride, debug=False):

        super(ResNet1D,self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.debug = debug
    

        # commonly used relu
        self.relu = nn.ReLU()

        # 1x1
        self.conv1_1x1 = nn.Conv1d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=kernel_size, stride=stride, padding=0, bias=False )
        self.batchnorm1 = nn.BatchNorm1d(self.out_channels)

    def forward(self,x):
        # input stored to be added before the final relu
        in_x = x

        # conv1x1->BN->relu
        x = self.conv1_1x1(x)
        if self.debug:
            print(f"[ResNet1D/forward] after CONV shape of x {x.shape}")
        
        x = self.batchnorm1(x)
        if self.debug:
            print(f"[ResNet1D/forward] after BatcNorm shape of x {x.shape}")
        
        x = self.relu(x)
        if self.debug:
            print(f"[ResNet1D/forward] after RELU shape of x {x.shape}")


        # identity with trucation
        in_x_truncated = in_x[...,:x.shape[2]]
        x_out = x + in_x_truncated

        # final relu
        x_out_after_relu = self.relu(x_out)
        
        return x_out_after_relu

# Dernière étape du réseau de Neurones
* Convolution passage de 300 à 10 feature maps
* et moyenne des données sur chaque couche (feature map) 
* après cette étape ce sera un softmax
## lien
* plus d'explications sur cette partie [lien vers le Markdown d'explication](https://github.com/javaskater/rcp217_project/blob/main/docs/PYTHON_STEPS/4-AVERAGING.md)
* [le code du module standalone avec une partie main de test](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/averaging_module.py)

In [ ]:
class AVERAGING(nn.Module):

    def __init__(self, debug=False):

        super(AVERAGING,self).__init__()
    
        self.debug = debug

        # commonly used relu
        self.relu = nn.ReLU()

        # 1x1
        self.conv1_before_averging = nn.Conv1d(in_channels=300, out_channels=10, kernel_size=10, stride=1, padding=0, bias=False )
        self.batchnorm_before_averging= nn.BatchNorm1d(10)

    def forward(self,x):

        # conv1x1->BN->relu
        x = self.conv1_before_averging(x)
        x = self.batchnorm_before_averging(x)
        x = self.relu(x)
        if self.debug:
            print(f"[AVERAGING/forward] after CONV/NORM/RELU shape of x {x.shape}")
        
        x = torch.mean(x, 2) # calculate the mean on the last dimension (batch size, number of )
        if self.debug:
            print(f"[AVERAGING/forward] after mean operation shape of x {x.shape}")

        return x

# Le réseau dans son ensemble
* c'est le réseau complet sous forme de nn.Module Pytorch (comme les 2 modules précédents), nom de la classe représentant le réseau **Project16**
* Fait appel au 2 modules Pytorch précédents
# liens
* plus d'explications sur cette partie [lien vers le Markdown d'explication](https://github.com/javaskater/rcp217_project/blob/main/docs/PYTHON_STEPS/5-GLOBAL.md)
* [le code du module standalone avec une partie main de test](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/projetct16_module.py)

In [ ]:
class Project16(nn.Module):

    def __init__(self, debug=False):

        super(Project16,self).__init__()

        self.debug = debug

        # commonly used relu
        self.relu = nn.ReLU()

        # from 1 time serie to 300 series of length 100 each 
        self.conv1_init = nn.Conv1d(in_channels=1, out_channels=300, kernel_size=10, stride=10, padding=0, bias=False)

        # 4 RESNET 1D with kernel of 10
        self.resnet10_1 = ResNet1D(300, 300, 10, 1, debug=debug)
        self.resnet10_2 = ResNet1D(300, 300, 10, 1, debug=debug)
        self.resnet10_3 = ResNet1D(300, 300, 10, 1, debug=debug)
        self.resnet10_4 = ResNet1D(300, 300, 10, 1, debug=debug)

        # 6 RESNET 1D with kernel of 1
        self.resnet1_1 = ResNet1D(300, 300, 1, 1, debug=debug)
        self.resnet1_2 = ResNet1D(300, 300, 1, 1, debug=debug)
        self.resnet1_3 = ResNet1D(300, 300, 1, 1, debug=debug)
        self.resnet1_4 = ResNet1D(300, 300, 1, 1, debug=debug)
        self.resnet1_5 = ResNet1D(300, 300, 1, 1, debug=debug)
        self.resnet1_6 = ResNet1D(300, 300, 1, 1, debug=debug)

        # The last averaging module
        self.averaging = AVERAGING(debug=self.debug)

    def forward(self,x):

        # conv1x1->BN->relu
        x = self.conv1_init(x)
        # first relu
        x = self.relu(x)
        if self.debug:
            print(f"[Project16/forward] after first convolution and relu shape of x {x.shape}")
        
        # 4 RESNET 1D with kernel of 10
        x = self.resnet10_1(x)
        x = self.resnet10_2(x)
        x = self.resnet10_3(x)
        x = self.resnet10_4(x)
        if self.debug:
            print(f"[Project16/forward] after 4 RESNET1D Blocks of kernel 10 shape of x {x.shape}")
        
        # 6 RESNET 1D with kernel of 1
        x = self.resnet1_1(x)
        x = self.resnet1_2(x)
        x = self.resnet1_3(x)
        x = self.resnet1_4(x)
        x = self.resnet1_5(x)
        x = self.resnet1_6(x)
        if self.debug:
            print(f"[Project16/forward] after 6 RESNET1D Blocks of kernel 1 shape of x {x.shape}")

        # from 300 to 10 feature maps and Averaging each map and outputting 10 softmax values
        x = self.averaging(x)
        x = TF.softmax(x,dim=1)
        if self.debug:
            print(f"[Project16/forward] after Averaging and softmax Blocks shape of x {x.shape}")
        
        return x

# Lancement de l'entrainement
* le code reprend [ce code Python à lancer dans notre environnement virtualisé](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/training_module.py) 
## TODO
* résoudre le problème de label à savoir comment le mettre en *Pytorch Tensor* cf. [la réponse 4 de ce Post Stackoverflow](https://stackoverflow.com/questions/63825841/attributeerror-tuple-has-no-attribute-to)
  * soit comme entier simple (de 0 à 9)
  * soit comme Tensor one hot vector
## lien vers la documentation
* C'est [le même lien que précédemment vers la documentation avec les sorties en erreur à corriger]()

In [ ]:
# From the Manning book's source code https://github.com/deep-learning-with-pytorch/dlwpt-code/blob/master/p1ch8/1_convolution.ipynb
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, device, debug=False):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        for series, labels in train_loader:  # <3>
            if debug == True:
                print(f"[training_loop] series batch before unsqueeze {series.shape}")                
            series = series.unsqueeze(1) # add the one channel dimension betwwen the batch_size dimension and the serie's length
            if debug == True:
                print(f"[training_loop] series batch after unsqueeze {series.shape}") 
            series = series.to(device)
            labels = labels.squeeze(1).long() # cf erreur 3 et 4 dans le compte rendu
            labels = labels.to(device=device)
              
            
            outputs = model(series)  # <4>
            
            if debug == True:
                print(f"[training_loop] outputs shape {outputs.shape} labels shape {labels.shape}")

            loss = loss_fn(outputs,labels)  # <5>

            optimizer.zero_grad()  # <6>
            
            loss.backward()  # <7>
            
            optimizer.step()  # <8>

            loss_train += loss.item()  # <9>

        if epoch == 1 or epoch % 5 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # <10>

data_train = TimeSeriesDatasetForPCoefficients(['/home/jpmena/CONSULTANT/CNAM/rcp217_project/R/ts_generees_28082025'])
train_loader = DataLoader(data_train, batch_size=50, shuffle=True)
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")
model = Project16().to(device=device)  # We use the Datanetwork just created 
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 50,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    device = device,
    debug=False
)

# La validation
* Dans cette partie, on teste le pourcentage de données exactes pour le paramètre p
  * et sur les données qui on servi pour l'entrainement (train)
  * et sur les données mises de côté pour la validation (test) 
## TODO
* pas encore testé
* prendre des données différentes de la validation (faire un nouveau lancement de la génération des Time Series)
## liens
* [le code en standalone (environnement local virtualisé)](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/validation_module.py)
  * noter que l'on recharge les coefficients du modèle sauvegardés suite au [lancement du training en environnement local virtualisé en Stand Alone](https://github.com/javaskater/rcp217_project/blob/main/PYTHON/training_module.py) 
* [Les explications et remarques sont en fin de ce MArkdown](https://github.com/javaskater/rcp217_project/blob/main/docs/PYTHON_STEPS/5-GLOBAL.md#validation-todo)

In [ ]:
# From the Manning book's source code https://github.com/deep-learning-with-pytorch/dlwpt-code/blob/master/p1ch8/1_convolution.ipynb
def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for series, labels in loader:
                series = series.to(device=device)
                labels = torch.tensor(labels).squeeze(1).long().to(device=device) # cf erreur 3 et 4 dans le compte rendu
                series2D = series.unsqueeze(1) # add the one channel dimension between the batch_size dimension and the serie's length
                outputs = model(series2D) 
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

data_path = '../R/' # the coeeficients are at the same level as my time series
data_train = TimeSeriesDatasetForPCoefficients([data_path + 'ts_generees_28082025'])
train_loader = DataLoader(data_train, batch_size=50, shuffle=True)
# TODO prendre une série générée différente
data_val = TimeSeriesDatasetForPCoefficients([data_path + 'ts_generees_28082025'])
val_loader = DataLoader(data_val, batch_size=50, shuffle=True)
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

loaded_model = model

optimizer = optim.SGD(loaded_model.parameters(), lr=1e-2)  #  <3>

loss_fn = nn.CrossEntropyLoss()  #  <4>

# we launch the validation on the training set and the validation set only to compare
all_acc_dict = collections.OrderedDict()
all_acc_dict["baseline"] = validate(loaded_model, train_loader, val_loader)